<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/python-code-for-FinancialAnalysis-book/blob/master/2020_6_1_chap4_numeric_summery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [ ]:
#Date: June 8, 2020
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew
from tabulate import tabulate
import datetime as dt#get current date
from google.colab import files

In [ ]:
class Analyze_Stock:

  def __init__(self, Stock_Name, Start_T, End_T, T_Interval):

    self.__Stock_price = yf.download(Stock_Name,start=Start_T,End=End_T,interval = T_Interval)['Adj Close'].dropna()
    self.__Ret = self.__Stock_price.pct_change()[1:]

    monthly_data = yf.download(Stock_Name,start=Start_T,End=End_T,interval = '1mo')['Adj Close'].dropna()
    slice_object = slice(1, len(monthly_data), 12)

    self.__Yr_Ret = monthly_data[slice_object].pct_change()[1:]
    self.__count = len(self.__Stock_price)
    self.stock = Stock_Name
    self.start = Start_T
    self.end = End_T
    self.frequency = T_Interval
  def __helper(self,N):
    return '{:0.3%}'.format(N)

  def __helper_2(self,N):
    return '{:0.2f}'.format(N)

  def __helper_4(self,N):
    return '{:0.4}'.format(N)

  def Mean_Ret(self):
    mean = np.mean(self.__Ret)
    return mean

  def Median_Ret(self):
    median = np.median(self.__Ret)
    return median

  def STD_Ret(self):
    std = np.std(self.__Ret)
    return std

  def Var_Ret(self):
    Var = np.var(self.__Ret)
    return Var

  def Annual_Mean_Ret(self):
    mean = np.mean(self.__Yr_Ret)
    return mean
  
  def Annual_Median_Ret(self):
    median = np.median(self.__Yr_Ret)
    return median

  def Annual_Std(self):
    std = np.std(self.__Yr_Ret)
    return std

  def Annual_Var(self):
    var = np.var(self.__Yr_Ret)
    return var

  def Min_Ret(self):
    Min = min(self.__Yr_Ret)
    return Min

  def Max_Ret(self):
    Max = max(self.__Yr_Ret)
    return Max

  def Range_Ret(self):
    Range = max(self.__Yr_Ret)-min(self.__Yr_Ret)
    return Range

  def Min_Price(self):
    Min = min(self.__Stock_price)
    return Min

  def Max_Price(self):
    Max = max(self.__Stock_price)
    return Max

  def Range_Price(self):
    Range = max(self.__Stock_price)-min(self.__Stock_price)
    return Range

  def quarter_tile(self):
    quarter = np.quantile(self.__Ret,0.25)
    return quarter

  def half_tile(self):
    half = np.quantile(self.__Ret,0.5)
    return half

  def Tquarter_tile(self):
    Tquarter = np.quantile(self.__Ret,0.75)
    return Tquarter
  
  def interquatile(self):
    Int_quarter = np.quantile(self.__Ret,0.75)-np.quantile(self.__Ret,0.25)
    return Int_quarter
  
  def kurt(self):
    Kurt = kurtosis(self.__Stock_price)
    return Kurt

  def skewness(self):
    Skew = skew(self.__Stock_price)
    return Skew

  def get_count(self):
    return self.__count
  
  def CoV(self):
    cov = np.std(self.__Ret)-np.mean(self.__Ret)
    return cov

##calculate correlation coefficient
  def Market_corr(self):
    SNP = yf.download('^GSPC',start=self.start,End=self.end,interval = self.frequency)['Adj Close'].dropna()
    corr = np.corrcoef(SNP,self.__Stock_price)
    return corr[0,1]

  def Get_summary(self):
    dict_data = {
    'Measures' :['Start Date','End Date','Frequency','Mean Return', 'Median Return', 'Standard deviation', 'Variance',\
                 'Annual Mean Return','Median Annual Return','Annual Standard Deviation', \
                  'Annual Variance','Minimum Return','Maximum Return','Range Return','Minimum Price',\
                 'Maximum Price','Range Price','Quatile 25%','Quatile 50%','Quatile 75%','Interquartile',\
                  'Kurtosis','Skewness','Count','Coefficient of Variation','Market corr with S&P 500'],
    self.stock : [self.start, self.end,self.frequency, self.__helper(self.Mean_Ret()),self.__helper(self.Median_Ret()),\
                  self.__helper(self.STD_Ret()),self.__helper(self.Var_Ret()),self.__helper(self.Annual_Mean_Ret()),\
                  self.__helper(self.Annual_Median_Ret()),self.__helper(self.Annual_Std()),self.__helper(self.Annual_Var()),\
                  self.__helper(self.Min_Ret()),self.__helper(self.Max_Ret()),self.__helper(self.Range_Ret()),self.__helper_2(self.Min_Price()),\
                  self.__helper_2(self.Max_Price()),self.__helper_2(self.Range_Price()),self.__helper(self.quarter_tile()),self.__helper(self.half_tile()),\
                  self.__helper(self.Tquarter_tile()),self.__helper(self.interquatile()),self.__helper_4(self.kurt()),\
                  self.__helper_4(self.skewness()),self.get_count(),self.__helper_4(self.CoV()),self.__helper_4(self.Market_corr())]
    }
    frame_data = pd.DataFrame(dict_data)
    return frame_data

In [ ]:
##allow user to input start&end date and interval
def Usr_input():
  #initialize dates
  previousday = dt.date.today()-dt.timedelta(days=1)# get the date of previous day
  Default_end = previousday.strftime("%Y-%m-%d") # change the format of the default datetime 
  years_ago = dt.date.today()-dt.timedelta(days=5 * 365)
  Default_start = years_ago.strftime("%Y-%m-%d")

  # request user input
  # set default values if users does not enter a value
  Start = input('Enter start date(' + 'Default = 5 Years ago, ' + Default_start + ')-->') or Default_start
  End = input('Enter end date(Default = Yesterday, ' + Default_end + ')-->') or Default_end
  T_interval = input('Enter date interval(1d,1wk,1mo Default = 1mo)-->') or '1mo'
  
  print("")
  return Start, End, T_interval

##allow user input the stock name
def Stock_input():
  Stock = []
  Stock.append(input('Please enter ticker '+str(1)+'(default be AAPL)-->') or 'AAPL') # use or operation the set default input
  Stock.append(input('Please enter ticker '+str(2)+'(default be MSFT)-->') or 'MSFT')
  return Stock

##return the summary of stock report
def get_all_summary():

  stocks=Stock_input()

  (macro_start, macro_end, macro_T_interval) = Usr_input()

##get report from SNP market report
  Market_ind = Analyze_Stock('^GSPC',macro_start,macro_end,macro_T_interval)
  M_summary = Market_ind.Get_summary()

  summary_append = []
  summary_append.append(M_summary)
##append stock report to it
  for i in range(2):
    test = Analyze_Stock(stocks[i],macro_start,macro_end,macro_T_interval)
    summary = test.Get_summary()
    summary_append.append(summary[test.stock])

  summary_append = pd.concat(summary_append,axis=1)
  return summary_append,stocks,macro_start, macro_end, macro_T_interval

In [ ]:
#Generate summary measure report
#ask for user input
(Total_summary,stocks,macro_start, macro_end, macro_T_interval) = get_all_summary()

##insert sp500 to stocks
stocks.insert(0,'^GSPC')

print("")
print('             Numerical Summary Measures of Rate of Returns')
print(' ')
print(tabulate(Total_summary, showindex=False, headers=Total_summary.columns,tablefmt="github"))

##download file to local

Please enter ticker 1(default be AAPL)-->
Please enter ticker 2(default be MSFT)-->
Enter start date(Default = 5 Years ago, 2015-07-11)-->
Enter end date(Default = Yesterday, 2020-07-08)-->
Enter date interval(1d,1wk,1mo Default = 1mo)-->

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

             Numerical Summary Measures of Rate of Returns
 
| Measures                  | ^GSPC      | AAPL       | MSFT       |
|-------

In [ ]:
Total_summary.to_csv('Chapter4_Summary.csv') 
files.download('Chapter4_Summary.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#print dataset
pd.set_option('display.max_rows', None)
data = yf.download(stocks,start=macro_start,End=macro_end,interval = macro_T_interval)['Adj Close'].dropna()
data = data[[stocks[0],stocks[1],stocks[2]]]#change arrangement of columns
data.columns = [stocks[0],stocks[1],stocks[2]] #display short names for index
print("")
print(tabulate(data, showindex=True, headers=data.columns,tablefmt="rst"))

##download file to local

[*********************100%***********************]  3 of 3 completed

===================  =======  ========  ========
..                     ^GSPC      AAPL      MSFT
===================  =======  ========  ========
2015-08-01 00:00:00  1972.18  104.068    39.3866
2015-09-01 00:00:00  1920.03  102.114    40.3205
2015-10-01 00:00:00  2079.36  110.631    47.9546
2015-11-01 00:00:00  2080.41  109.52     49.5124
2015-12-01 00:00:00  2043.94   97.8647   50.8825
2016-01-01 00:00:00  1940.24   90.5012   50.5248
2016-02-01 00:00:00  1932.23   89.8969   46.6637
2016-03-01 00:00:00  2059.74  101.883    51.0169
2016-04-01 00:00:00  2065.3    87.627    46.0658
2016-05-01 00:00:00  2096.95   93.348    48.957
2016-06-01 00:00:00  2098.86   89.9098   47.5972
2016-07-01 00:00:00  2173.6    98.0074   52.7225
2016-08-01 00:00:00  2170.95   99.7849   53.448
2016-09-01 00:00:00  2168.27  106.897    53.9122
2016-10-01 00:00:00  2126.15  107.36     56.0837
2016-11-01 00:00:00  2198.81  104.505    56.4019
2

In [ ]:
data.to_csv('Chapter4_Summary_data.csv') 
files.download('Chapter4_Summary_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
a = Analyze_Stock('AAPL','2010-11-23','2012-11-23','1d')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
